In [1]:
 # THE FOLLOWING ARE LIBRARIES YOU NEED
# You have to do install
#    tensorflow, tflearn, opencv-python, tqdm

In [10]:
# IMPORT STATEMENTS, AND SETTING UP VARS
from tqdm import tqdm
import os
import cv2
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, image_preloader
from tflearn.metrics import Accuracy
from time import time
 
import tensorflow as tf
tf.compat.v1.reset_default_graph()
acc = Accuracy()

# https://www.geeksforgeeks.org/image-classifier-using-cnn/

TRAIN_DIR = "Alzheimers_Dataset_V1/train"
TEST_DIR = "Alzheimers_Dataset_V1/test"
IMG_SIZE = 50 # experiment on this
LR = 1e-3
MODEL_NAME = "classifying_alzheimers_v1.model"

train_images = "train_images.txt"
test_images = "test_images.txt"

# x_image = tf.compat.v1.placeholder("float", shape=[None, img_size, img_size, 1])
# y_ = tf.compat.v1.placeholder("float", shape=[None, 2])

In [3]:
# FOR TRAINING DATA, TELLS THE DATA WHICH OF THE 4 CATEGORIES THE IMAGE BELONGS TO
# def label_img(folder):
#     if folder == 'NonDemented': return [1, 0, 0, 0]
#     elif folder == 'VeryMildDemented': return [0, 1, 0, 0]
#     elif folder == 'MildDemented': return [0, 0, 1, 0]
#     elif folder == 'ModerateDemented': return [0, 0, 0, 1]
def label_img(folder):
    if folder == 'NonDemented': return 0
    elif folder == 'VeryMildDemented': return 1
    elif folder == 'MildDemented': return 2
    elif folder == 'ModerateDemented': return 3

In [14]:
# GOES THROUGH EACH FOLDER, GOES THROUGH EACH IMAGE, GETS THE IMAGE, AND MAKES THE TRAINING DATA
def create_train_data():
    training_data = []
    for folder in tqdm(os.listdir(TRAIN_DIR), disable=True):
        label = label_img(folder)
        if folder == ".DS_Store": continue
        fol_path = os.path.join(TRAIN_DIR, folder)
        
        for img in tqdm(os.listdir(fol_path), disable=True):
            img_path = os.path.join(fol_path, img)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                training_data.append([np.array(img), np.array(label)])
            except:
                pass
    shuffle(training_data)
    # print(training_data)
    # saving our trained data for further uses if required
    np.save('train_data.npy', training_data)
    return training_data

In [15]:
# GOES THROUGH EACH FOLDER, GOES THROUGH EACH IMAGE, GETS THE IMAGE, AND MAKES THE TESTING DATA
def process_test_data():
    test_data = []
    for folder in tqdm(os.listdir(TEST_DIR), disable=True):
        if folder == ".DS_Store": continue
        fol_path = os.path.join(TEST_DIR, folder)
        label = label_img(folder)
        for img in tqdm(os.listdir(fol_path), disable=True):
            img_num = os.listdir(fol_path).index(img)
            img_path = os.path.join(fol_path, img)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                test_data.append([np.array(img), np.array(label)])
            except:
                pass
    shuffle(test_data)
    # print(training_data)
    # saving our trained data for further uses if required
    np.save('test_data.npy', test_data)
    return test_data

In [48]:
# GETTING THE DATA
start = int(round(time()*1000))
train_data = create_train_data()
test_data = process_test_data()
end = int(round(time()*1000))
getting_time = (end-start)/1000
print("That took %s mins (%s seconds)" % (getting_time/60, getting_time))
# x_train, y_train = image_preloader(train_images, image_shape=(img_size, img_size), mode='file', categorical_labels=True, grayscale=True)
print("There are %s images total: %s images in train[], and %s images in test[]"
     % (len(train_data) + len(test_data), len(train_data), len(test_data)))
# train_data = np.load('train_data.npy', allow_pickle=True)
# test_data = np.load('test_data.npy', allow_pickle=True)

# np.shape(y_train)

That took 0.08953333333333333 mins (5.372 seconds)
There are 6400 images total: 5121 images in train[], and 1279 images in test[]


In [49]:
# FOR TRAINING PURPOSES, IT SPLITS THE TRAINING DATA INTO 2
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = np.array([i[1] for i in train])
# because the data is formatted by [img, which category], that makes it so that
# x = img, and y = which category
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = np.array([i[1] for i in test])
# same for this

In [57]:
# TENSORFLOW SETUP (I think..?)
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input')
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 128, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = fully_connected(convnet, 1024, activation ='tanh')
convnet = dropout(convnet, 0.8)
 
convnet = fully_connected(convnet, 4, activation ='softmax')
convnet = regression(convnet, optimizer ='adam', learning_rate = LR,
      loss ='categorical_crossentropy', name ='targets', n_classes=4, metric=acc)

model = tflearn.DNN(convnet, tensorboard_dir ='log')

In [58]:
convnet


<tf.Tensor 'FullyConnected_15/Softmax:0' shape=(?, 4) dtype=float32>

In [59]:
# TRAINING THE MODEL
start = int(round(time()*1000))
model.fit({'input': X}, {'targets': Y}, n_epoch = 100,
    snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)
model.save(MODEL_NAME)
end = int(round(time()*1000))
training_time = (end-start) / 1000
print("Training time: %s mins (%s seconds)" % (training_time/60, training_time))

---------------------------------
Run id: classifying_alzheimers_v1.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy_6/Adam_0 (raw) is illegal; using Accuracy_6/Adam_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_7/Adam_1 (raw) is illegal; using Accuracy_7/Adam_1__raw_ instead.
---------------------------------
Training samples: 36968
Validation samples: 0
--


InvalidArgumentError: You must feed a value for placeholder tensor 'input_1/X' with dtype float and shape [?,50,50,1]
	 [[node input_1/X
 (defined at /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tflearn/layers/core.py:81)
]]

Errors may have originated from an input operation.

Operation defined at: (most recent call last)
>>>   File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_in

In [30]:
import matplotlib.pyplot as plt
test_data = np.load('test_data.npy', allow_pickle=True)
fig = plt.figure()
score = 0
for data in test_data:
    img_num = data[1]
    img_data = data[0]

    orig = img_data
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)

    model_out = model.predict_label([data])[0]

    if model_out[0] == img_num:
        score += 1
accuracy = score/len(test_data)
print("Score: %s" % accuracy)